In [3]:
from uszipcode import SearchEngine
import networkx as nx
import pickle
import math
from area import area
import fiona
from shapely.geometry import shape
from pyproj import Proj, transform, Transformer
from shapely.geometry import Polygon
import folium as f
from scipy.spatial import ConvexHull

g_pickle = "01_cambridge-small.gpickle"
g = nx.read_gpickle(g_pickle)


search = SearchEngine(simple_zipcode=False, db_file_dir="./zip_code_cache")

In [4]:
shapes = fiona.open("./mass_zips/zipcodes_nt/ZIPCODES_NT_POLY.shp")
input_proj = shapes.crs.get('init')
output_proj = "epsg:4326"
transformer = Transformer.from_crs(input_proj, output_proj)

polygons = {}
error_zips = []
for shape in shapes:
    zip_code = shape.get('properties').get('POSTCODE')    
    points = []
    for coord in shape.get('geometry').get('coordinates')[0]:
        x = coord[0]
        y = coord[1]
        xy = transformer.transform(x,y)
        points.append([xy[1], xy[0]])
    try: 
        polygons[zip_code] = Polygon(points)
    except: error_zips.append(zip_code)
        
print("Errors with these zips: " + str(error_zips))

Errors with these zips: ['02360', '02790', '01450', '02535', '01742', '02568', '02554', '02649', '01938', '01844', '01826', '02667', '02050', '02539', '02536', '01930', '01463', '02777', '01585', '02748', '02332', '01966', '02481', '02655', '02420', '02043', '01852', '02740', '02559', '01529', '01905', '01945', '02630', '02571', '02066', '01969', '01951', '01879', '01701', '02563', '02739', '01821', '01050', '02668', '02653', '01778', '01760', '01952', '01982', '01915', '02719', '02642', '01923', '01929', '02713', '02738', '02186', '02169', '01803', '02633', '01504', '02718', '02025', '02673', '01862', '01950', '01008', '01835', '01944', '02148', '02472', '01854', '02675', '02155', '02045', '02191', '02543', '01380', '02453', '02125', '02127', '02670', '02534', '02744', '02129', '02026', '02532', '01908', '02791', '02641', '01965', '02210', '02647', '02116', '02215', '02462', '02467', '02114', '02163', '02671', '02467', '02152']


In [5]:
g = nx.read_gpickle("01_cambridge.gpickle")
nodes_data_subset = g.nodes().data()


### `zip_code_dict` and `nodes_by_zip` are inverses of each other

In [6]:
with open('zip_code_dict_NEW.pickle', 'rb') as handle:
    zip_code_dict = pickle.load(handle)

# Generate reverse dictionary——nodes by zip instead of zips by node
def get_nodes_by_zip(zip_code_dict):
    nodes_by_zip = {}
    for node_id, zip_code in zip_code_dict.items():
        nodes_by_zip.setdefault(zip_code, []).append(node_id)
    return nodes_by_zip


# zip_code_dict = get_zip_code_dict(g)
nodes_by_zip = get_nodes_by_zip(zip_code_dict)

In [7]:
def area_km2_from_g_nodes(g, zip_code):
    lat_lons = [[x[1]["lat"], x[1]["lon"]] for x in g.nodes().data() if x[0] in nodes_by_zip[zip_code]]
    if len(lat_lons) < 3:
        return .000000001
    hull = ConvexHull(lat_lons)
    hull_edges = [[hull.points[ix][1], hull.points[ix][0]] for ix in hull.vertices]
    polygon = {'type': 'Polygon', 'coordinates': [hull_edges]}
    return (area(polygon) / 1000000)

def gen_population_dict(g):
    zips_in_g = {zip_code_dict[node_id] for node_id in g}
    population_dict = {}
    nodes_in_g = set(g)
    
    entire_pop = {z: search.by_zipcode(z).population for z in zips_in_g}
    pop_density = {z: search.by_zipcode(z).population_density for z in zips_in_g}
    land_area_in_km2 = {z: (search.by_zipcode(z).land_area_in_sqmi * 2.58999) for z in zips_in_g}
    nodes_in_zip = {z: list(set(nodes_by_zip[z]) & nodes_in_g) for z in zips_in_g}
    convex_hull_in_km2 = {z: area_km2_from_g_nodes(g, z) for z in zips_in_g}
    node_area_percent = {z: min([1, convex_hull_in_km2[z]/land_area_in_km2[z]]) for z in zips_in_g}
    est_pop_in_convex_hull = {z: area_km2_from_g_nodes(g, z) for z in zips_in_g}
    
    
    
    return  {z: {
                 "entire_pop": entire_pop[z],
                 "pop_density": pop_density[z],
                 "land_area_in_km2": land_area_in_km2[z],
                 "land_area_from_nodes": convex_hull_in_km2[z],
                 "node_area_percent": node_area_percent[z],
                 "nodes": nodes_in_zip[z],
                 "pop_in_node_area": math.ceil(entire_pop[z] * node_area_percent[z])
                } 
              for z in zips_in_g}
    
    
pop_dict = gen_population_dict(g)

pop_dict

KeyError: '61171503'

# View zip codes and associated points

In [9]:
ZIP_CODE = '02138'
m = f.Map(location = [42.3611108,-71.119977], zoom_start=14)


style = {
    "fillColor": "salmon",
    "fillOpacity": .2,
    "weight": 2,
    "opacity": 1
}

with open('zip_code_dict_NEW.pickle', 'rb') as handle:
    zip_code_dict = pickle.load(handle)
print(len(zip_code_dict))

# Show graph with hull edges
for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    if node_id in nodes_by_zip[ZIP_CODE]:
        if not node_data[1].get('lon'):
            import pdb; pdb.set_trace()
        lon,lat = node_data[1]['lon'], node_data[1]['lat'] 
#         if [lon, lat] in hull_edges:
#             m.add_child(f.Marker(location=[lat,lon], color="red", radius=1))
        m.add_child(f.CircleMarker(location=[lat,lon], color="blue", radius=1))

        
f.GeoJson(polygons[ZIP_CODE], style_function=lambda x: style).add_to(m)

m


77404


In [59]:
from uszipcode import SearchEngine
import networkx as nx
import pickle
# import itertools
# import random
import math
from area import area
import fiona
from shapely.geometry import shape
from pyproj import Proj, transform, Transformer
# from uszipcode import SearchEngine
# from shapely.geometry import Polygon, Point
# import networkx as nx
# import random
# import pickle
# import math
from shapely.geometry import Polygon
# import shapely.geometry as geometry
import folium as f
from scipy.spatial import ConvexHull
# from uszipcode import SearchEngine
# import networkx as nx
# import pickle
# import itertools
# import random
# import pyproj
# from area import area
# import math
# from shapely.geometry import Polygon
# import shapely.geometry as geometry
# import folium as f
# from scipy.spatial import ConvexHull
# from uszipcode import SearchEngine
# from shapely.geometry import Polygon, Point
# import networkx as nx
# import random
# import pickle

## This is useful scratch, but mainly cruft. Use `area` library to find Polygon's geographic area

### First, simple PoC with Wyoming

In [ ]:
wyoming = {'type':'Polygon','coordinates':[[[-111.046768, 40.997963], 
                                            [-111.055196, 45.001320], 
                                            [-104.057691, 44.997377],
                                            [-104.053251, 41.001410],
                                            [-111.046768, 40.997963]]]}

area_km2 = area(wyoming)

area_km2 = area_km2 / 1e+6
# print ('area m2: ' + str(math.floor(area_m2)))
print ('Estimated area of Wyoming: ' + str(math.floor(area_km2)) + 'km^2')
print("Error:                     " + str(253600 - math.floor(area_km2)) + "km^2" )

### Now running on entire Cambridge dataset. (Note—outliers skew this)

In [ ]:
from scipy.spatial import ConvexHull

lat_lons = [[x[1]["lat"], x[1]["lon"]] for x in g.nodes().data()]
hull = ConvexHull(lat_lons)
hull_edges = [[hull.points[ix][1], hull.points[ix][0]] for ix in hull.vertices]

camb = {'type': 'Polygon', 'coordinates': [hull_edges]}

print("Area of convex map: " + (str (area(camb) / 1000000)) + "km^2")



In [ ]:
m = f.Map(location = [42.3611108,-71.1079923], zoom_start=16)

# Show graph with hull edges
for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    if not node_data[1].get('lon'):
        import pdb; pdb.set_trace()
    lon,lat = node_data[1]['lon'], node_data[1]['lat']  
    if [lon, lat] in hull_edges:
        m.add_child(f.Marker(location=[lat,lon], color="red", radius=1))
    
    m.add_child(f.CircleMarker(location=[lat,lon], color="orange", radius=.5))
    
    

m

# *_CRUFT ZONE_* (mostly noodling trying to get area from nodes)

In [ ]:
import pyproj    
import shapely
import shapely.ops as ops
from shapely.geometry.polygon import Polygon
from functools import partial


# geom = Polygon([[40.997963, -111.046768], 
#                 [45.001320, -111.055196], 
#                 [44.997377, -104.057691],
#                 [41.001410, -104.053251],
#                 [40.997963, -111.046768]])


geom = Polygon([[-111.046768, 40.997963], 
                [-111.055196, 45.001320], 
                [-104.057691, 44.997377],
                [-104.053251, 41.001410],
                [-111.046768, 40.997963]])
# geom = Polygon([[40,-111],
#                 [2,3],
#                 [3,5],
#                 [40,-111]])
geom_area = ops.transform(
    partial(
        pyproj.transform,
        pyproj.Proj(init='EPSG:4326'),
        pyproj.Proj(
            proj='aea',
            lat_1=geom.bounds[1],
            lat_2=geom.bounds[3])),
    geom)

# Print the area in m^2
print (math.floor(geom_area.area / 1000000) )

In [ ]:
ZIP_CODE = '02139'
search.by_zipcode(ZIP_CODE)

In [ ]:
search.by_zipcode(ZIP_CODE).polygon

### `alphashape` is too expensive and requires too much noodling to be of use for us for this problem

In [ ]:


# import alphashape

# points = [(17, 158),(15, 135),(38, 183),(43, 19),(93, 88),(96, 140),(149, 163),(128, 248),(216, 265),(248, 210),(223, 167),(256, 151),(331, 214),(340, 187),(316, 53),(298, 35),(182, 0),(121, 42)]
# lat_lons = [[x[1]["lat"], x[1]["lon"]] for x in g.nodes().data()]
# points = lat_lons[:1000]
# alpha = 0.95 * alphashape.optimizealpha(points)
# hull = alphashape.alphashape(points, alpha)
# hull_pts = hull.exterior.coords.xy
# print(hull_pts)

# from scipy.spatial import ConvexHull

# lat_lons = [[x[1]["lat"], x[1]["lon"]] for x in g.nodes().data()]
# hull = ConvexHull(lat_lons)
# hull_edges = [[hull.points[ix][1], hull.points[ix][0]] for ix in hull.vertices]

# camb = {'type': 'Polygon', 'coordinates': [hull_edges]}

# print("Area of convex map: " + (str (area(camb) / 1000000)) + "km^2")



In [ ]:




from shapely.ops import cascaded_union, polygonize
from scipy.spatial import Delaunay
from shapely.geometry import Point

import numpy as np
import geopandas

import math
def alpha_shape(points, alpha):
    """
    Compute the alpha shape (concave hull) of a set
    of points.
    @param points: Iterable container of points.
    @param alpha: alpha value to influence the
        gooeyness of the border. Smaller numbers
        don't fall inward as much as larger numbers.
        Too large, and you lose everything!
    """
    if len(points) < 4:
        # When you have a triangle, there is no sense
        # in computing an alpha shape.
        return geometry.MultiPoint(list(points)).convex_hull
    def add_edge(edges, edge_points, coords, i, j):
        """
        Add a line between the i-th and j-th points,
        if not in the list already
        """
        if (i, j) in edges or (j, i) in edges:
            # already added
            return
        edges.add( (i, j) )
        edge_points.append(coords[ [i, j] ])
    coords = np.array([point.coords[0]
                       for point in points])
    tri = Delaunay(coords)
    edges = set()
    edge_points = []
    # loop over triangles:
    # ia, ib, ic = indices of corner points of the
    # triangle
    for ia, ib, ic in tri.vertices:
        pa = coords[ia]
        pb = coords[ib]
        pc = coords[ic]
        # Lengths of sides of triangle
        a = math.sqrt((pa[0]-pb[0])**2 + (pa[1]-pb[1])**2)
        b = math.sqrt((pb[0]-pc[0])**2 + (pb[1]-pc[1])**2)
        c = math.sqrt((pc[0]-pa[0])**2 + (pc[1]-pa[1])**2)
        # Semiperimeter of triangle
        s = (a + b + c)/2.0
        # Area of triangle by Heron's formula
        area = math.sqrt(s*(s-a)*(s-b)*(s-c))
        circum_r = a*b*c/(4.0*area)
        # Here's the radius filter.
        #print circum_r
        if circum_r < 1.0/alpha:
            add_edge(edges, edge_points, coords, ia, ib)
            add_edge(edges, edge_points, coords, ib, ic)
            add_edge(edges, edge_points, coords, ic, ia)
    m = geometry.MultiLineString(edge_points)
    triangles = list(polygonize(m))
    return cascaded_union(triangles), edge_points

concave_hull, edge_points = alpha_shape(([Point(x[0], x[1]) for x in [[11,2], [12,2], [22,1], [12.5,1.5], [12,21]]]),
                                        alpha=.2)

In [ ]:
concave_hull.area

In [ ]:
concave_hull

In [ ]:
set(g)

In [ ]:
def CRUFT_semi_random_nodes(g, percentage):
    k = percentage * len(g)
    zips_in_g = {zip_code_dict[node_id] for node_id in g}
    
    weight_by_zip = {}
    for zip_code in zips_in_g:
        weight_by_zip[zip_code] = area_km2_from_map(g, zip_code) * search.by_zipcode(zip_code).population_density
    
    sum_of_weights = sum([v for k,v in weight_by_zip.items()])    
    set_of_nodes_g = set({node for node in g})
    
    num_by_zip = {}
    for zip_code in zips_in_g:
        absolute_num = math.ceil(weight_by_zip[zip_code]*k/sum_of_weights)
        nodes_in_zip_code = list(set(nodes_by_zip[zip_code]) & set_of_nodes_g)
        if absolute_num > len(nodes_in_zip_code):
            print(zip_code)
            print("absolute num:    " + str(absolute_num))
            print("number of nodes: " + str(len(nodes_in_zip_code)))
            area_from_points = (area_km2_from_map(g, zip_code))
            land_area = (search.by_zipcode(zip_code).land_area_in_sqmi * 2.58999)
            print(area_from_points / land_area)
            print("____")
        else: print("***" + str(absolute_num))
        num_by_zip[zip_code] = absolute_num
#     print("[[[[[[[[[[[[[[[]]]]]]]]]]]]]]]")
#     for zip_code in zips_in_g:
#         nodes_in_zip_code = list(set(nodes_by_zip[zip_code]) & set_of_nodes_g)

#         print(zip_code)
#         print("number of nodes: " + str(len(nodes_in_zip_code)))
        
    return_nodes = {}
    for zip_code,num in num_by_zip.items():
        nodes_in_zip_code = list(set(nodes_by_zip[zip_code]) & set_of_nodes_g)
        if len(nodes_in_zip_code) > num:
            return_nodes[zip_code] = random.sample(nodes_in_zip_code, num)
        else:    
            return_nodes[zip_code] = nodes_in_zip_code
    

          
    return list(itertools.chain.from_iterable([nodes for zip_code,nodes in return_nodes.items()]))
aa = semi_random_nodes(g, 1)
print(len(aa)/ len(g))
## ISSUE HERE: why does it go down down down when the k value goes up?

In [ ]:
[42.359811, -71.073183]

In [ ]:
# [z.zipcode for z in search.by_coordinates(42.359811, -71.073183, returns=50)]

In [ ]:
# polyg.contains(Point(-71.0825, 42.3636))


# for node_data in g.nodes().data():
#         node_id = node_data[1]["id"]
#         lon,lat = node_data[1]['lon'], node_data[1]['lat']
#         if polyg.contains(Point(lon, lat)):
#             print("hit")


In [ ]:
import fiona
from shapely.geometry import shape

from pyproj import Proj, transform
from pyproj import Transformer

shapes = fiona.open("./mass_zips/zipcodes_nt/ZIPCODES_NT_POLY.shp")
input_proj = shapes.crs.get('init')
output_proj = "epsg:4326"
transformer = Transformer.from_crs(input_proj, output_proj)

polygons = {}
for shape in shapes:
    zip_code = shape.get('properties').get('POSTCODE')    
    points = []
    for coord in shape.get('geometry').get('coordinates')[0]:
        x = coord[0]
        y = coord[1]
        xy = transformer.transform(x,y)
        points.append([xy[1], xy[0]])
    try: 
        polygons[zip_code] = Polygon(points)
    except: print(zip_code)

In [ ]:
zip_code_dict = {}

In [ ]:
g_pickle = "01_boston-area.gpickle"
g = nx.read_gpickle(g_pickle)
for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    lon,lat = node_data[1]['lon'], node_data[1]['lat']
    if node_id in zip_code_dict:
        continue
    for z in polygons:
        polygon = polygons[z]
        if polygon.contains(Point(lon,lat)):
            zip_code_dict[node_id] = z
            break


In [ ]:
len(g)

In [ ]:
len(zip_code_dict)

In [ ]:
with open('zip_code_dict_NEW.pickle', 'wb') as handle:
        pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL) 

In [ ]:
m = f.Map(location = [42.3611108,-71.119977], zoom_start=14)


style = {
    "fillColor": "salmon",
    "fillOpacity": .2,
    "weight": 1,
    "opacity": 1
}

# with open('zip_code_dict_NEW.pickle', 'rb') as handle:
#     zip_code_dict = pickle.load(handle)

# for node_data in g.nodes().data():
#     node_id = node_data[1]["id"]
#     if node_id in nodes_by_zip[ZIP_CODE]:
#         if not node_data[1].get('lon'):
#             import pdb; pdb.set_trace()
#         lon,lat = node_data[1]['lon'], node_data[1]['lat'] 
#         if [lon, lat] in hull_edges:
#             m.add_child(f.Marker(location=[lat,lon], color="red", radius=1))
#         m.add_child(f.CircleMarker(location=[lat,lon], color="blue", radius=1))
    

f.GeoJson(polygon, style_function=lambda x: style).add_to(m)

m

In [1]:
import csv
cccc = 1
with open('./data/boston-metro_zipcodes.csv', newline='') as csvfile:
    r = csv.reader(csvfile, delimiter=',')
    for row in r:
        cccc += 1
#         if row[0] == '30730968':
#             print(row[1])
#             break
cccc

308151